# Q1


In [12]:
# !wget -P '/home/codespace/mlops-zoomcamp/data/yellow_tripdata_2023-01.parquet' https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
# !wget -P '/home/codespace/mlops-zoomcamp/data/yellow_tripdata_2023-02.parquet' https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

In [13]:
%pip install 'dask[dataframe]'
# %pip install pyarrow

  Using cached dask_expr-1.1.16-py3-none-any.whl.metadata (2.5 kB)
  Using cached pyarrow-17.0.0.tar.gz (1.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached dask_expr-1.1.16-py3-none-any.whl (243 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [347 lines of output]
      <string>:34: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      running bdist_wheel
      running build
      running build_py
      creating build/lib.macosx-10.13-universal2-cpython-313/benchmarks
      copying benchmarks/array_ops.py -> build/lib.macosx-10.13-universal2-cpython-313/benchmarks
      copying benchmarks/convert_builtins.py -> build/lib.macosx-10.13-universal2-cpython-313/benchmarks
      copying benchmarks/parquet.py -> build/lib.macos

In [14]:
import pandas as pd

df_jan = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")


len(df_jan.head().columns)

19

# Q2

In [15]:
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
df_jan["duration"] = df_jan["duration"].map(lambda x: x.total_seconds() / 60)
std = df_jan["duration"].std()
print(f"std: {std:.2f}")

std: 42.59


# Q3



In [16]:
df_jan_no_outliers = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)]

percent_diff = (len(df_jan_no_outliers) / len(df_jan)) * 100
print(f"percent_diff: {percent_diff:.2f}%")

percent_diff: 98.12%


In [17]:
df_jan_no_outliers.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


# Q4

In [18]:
pu_do = ["PULocationID", "DOLocationID"]

df_jan_no_outliers[pu_do] = df_jan_no_outliers[pu_do].astype(object)
train_df = pd.get_dummies(df_jan_no_outliers[pu_do], sparse=True, dtype=int)

train_df.info()

/var/folders/7p/m0f5r_5s72g21m2zr560q7tc0000gn/T/ipykernel_88366/3712424095.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_no_outliers[pu_do] = df_jan_no_outliers[pu_do].astype(object)


<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Columns: 515 entries, PULocationID_1 to DOLocationID_265
dtypes: Sparse[int64, 0](515)
memory usage: 91.8 MB


: 

In [19]:
train_dicts = train_df.to_dict(orient="records")

In [11]:
from sklearn.feature_extraction import DictVectorizer


dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

NameError: name 'train_dicts' is not defined

# Q5



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

model = LinearRegression()
model.fit(X_train, df_jan_no_outliers["duration"])

y_pred = model.predict(X_train)
rmse = root_mean_squared_error(df_jan_no_outliers["duration"], y_pred)
print(f"rmse: {rmse:.2f}")

# Q6

In [ ]:
df_feb = pd.read_parquet("../data/yellow_tripdata_2023-02.parquet")

df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
df_feb["duration"] = df_feb["duration"].map(lambda x: x.total_seconds() / 60)

df_feb_no_outliers = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60)]

In [ ]:
pu_do = ["PULocationID", "DOLocationID"]

df_feb_no_outliers[pu_do] = df_feb_no_outliers[pu_do].astype(object)
test_df = pd.get_dummies(df_feb_no_outliers[pu_do], sparse=True, dtype=int)

In [ ]:
del df_feb

In [ ]:
test_dicts = test_df.to_dict(orient="records")

In [ ]:
X_test = dv.transform(test_dicts)

y_pred = model.predict(X_test)
rmse = root_mean_squared_error(df_feb_no_outliers["duration"], y_pred)
print(f"rmse: {rmse:.2f}")

In [ ]:
from pickletools import int4


int4

In [ ]:
df.